### with this script you can add attributes from raster or polygons to points 

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

import geopandas
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point

import ee
import geemap
import pandas as pd

In [2]:
# Initialize Earth Engine
ee.Initialize()

##### read points

In [3]:
points = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2023_clean.csv', delimiter=',', low_memory=False)
#read GEE points
#points = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_cmr')

In [4]:
all_columns = points.columns.tolist()

# Display the list of column names
print(all_columns)

['ID', 'plotid', 'sampleid', 'lon', 'lat', 'TNT_stratum', 'sample_geom', 'Ref_FNF_2022', 'Ref_Regeneration', 'Ref_Change_2023', 'Ref_Change_Type_2023', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_NFtype_2022', 'Ref_Ftype_2022', 'collection', 'interpreted', 'Ref_LCover_2022', 'Ref_Change_Year_2023', 'Ref_Change_LCover_2023', 'Def2023', 'Deg2023', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrove_Deg', 'Swamp_Deg', 'Gallery_Deg', 'Plantation_Deg', 'Woodland_Deg', 'DensFor_stable', 'DensDryFor_stable', 'SecFor_stable', 'DryOpenFor_stable', 'Mangr

In [19]:
#drop any columns if needed
#esbae = esbae.drop('index_right', axis=1)

In [5]:
#enter Lat and Lon columns
LATcol = 'lat'
LONcol = 'lon'

In [6]:
gdf = geopandas.GeoDataFrame(
    points, geometry=geopandas.points_from_xy(points[LONcol], points[LATcol]), crs="EPSG:4326")

#### add attributes from local shapefile

In [8]:
poly = "/home/sepal-user/data/admin/RDC_Province_26.shp"
#poly = "/home/sepal-user/data/drc/DRC_gadm2.shp"
#poly = "/home/sepal-user/data/other_data/CAFI_tnc_ecoregions.shp"
# Read file using gpd.read_file()
poly_shp = gpd.read_file(poly)
poly_shp.head()

,SCE_SEM,SCE_GEO,MODIF,ORIGINE,NOM,CODE_INS,SURFACE,CHEF_LIEU,ID_SEPAL,BLABLA,geometry
0,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin / Image satellite,Kinshasa,10,10584.151287,Kinshasa,1,0,"POLYGON ((1775596.089 -434954.674, 1775691.545..."
1,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Kongo Central,20,54493.327517,Matadi,2,0,"POLYGON ((1604421.487 -477541.359, 1605095.950..."
2,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Mai-Ndombe,302,128788.166437,Inongo,3,0,"POLYGON ((2132294.536 -90899.292, 2133985.101 ..."
3,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin / Image satellite,Kwilu,303,79747.494200,Bandundu,4,0,"POLYGON ((1936677.519 -353809.382, 1936960.249..."
4,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin / Image satellite,Kwango,305,91102.901904,Kenge,5,0,"POLYGON ((1960581.946 -468508.291, 1961890.275..."


In [9]:
poly_crs = poly_shp.crs
print("Current CRS:", poly_crs)

Current CRS: PROJCS["World_Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_2SP"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","54004"]]


In [10]:
poly_shp = poly_shp.to_crs(gdf.crs)

In [13]:
# Perform the spatial join using the predicate parameter
joined_data = gpd.sjoin(gdf, poly_shp, how="left", predicate="within", lsuffix='left', rsuffix='right')

# 'joined_data' now contains the points with additional information from the polygons

# Print or explore the resulting GeoDataFrame
joined_data.head()

,ID,plotid,sampleid,lon,lat,TNT_stratum,sample_geom,Ref_FNF_2022,Ref_Regeneration,Ref_Change_2023,...,SCE_SEM,SCE_GEO,MODIF,ORIGINE,NOM,CODE_INS,SURFACE,CHEF_LIEU,ID_SEPAL,BLABLA
0,0,0,0,19.592661,-2.323897,2,POINT(19.59266059 -2.323896724),1,NaN,0,...,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Mai-Ndombe,302,128788.166437,Inongo,3,0
1,1,1,1,23.356152,0.515229,2,POINT(23.35615247 0.515228731),1,NaN,0,...,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Tshopo,502,200560.036106,Kisangani,11,0
2,2,2,2,23.858580,-6.530078,2,POINT(23.85858021 -6.530078379),0,non,0,...,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Lomami,804,54613.078273,Kabinda,24,0
3,3,3,3,24.998003,-8.514008,2,POINT(24.99800332 -8.514007684),1,NaN,0,...,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Haut-Lomami,705,111374.971738,Kamina,19,0
4,4,4,4,23.009672,-0.680159,2,POINT(23.00967227 -0.680159417),1,NaN,0,...,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Tshuapa,407,132981.109451,Boende,10,0


In [14]:
joined_data['NOM'].value_counts(dropna=False)

NOM
Haut-Lomami       42
Lualaba           38
Tanganyka         36
Haut-Uele         32
Tshopo            29
Haut-Katanga      27
Sud-Kivu          24
Bas-Uele          24
Maniema           24
Kwilu             22
Mai-Ndombe        22
Kwango            21
Mongala           19
Tshuapa           17
Kasai             17
Nord-Ubangi       15
Equateur          15
Kasai Central     15
Lomami            14
Ituri             13
Sankuru           13
Nord-Kivu         13
Kongo Central     13
Sud-Ubangi         6
Kasai Oriental     3
Name: count, dtype: int64

In [18]:
province_pts = pd.pivot_table(joined_data,values='plotid',index=['NOM'],columns=['TNT_stratum'],aggfunc="count",margins=True,
                             margins_name='Total')
province_pts

TNT_stratum,2,3,4,5,Total
NOM,,,,,
Bas-Uele,12.0,5.0,4.0,3.0,24
Equateur,7.0,2.0,3.0,3.0,15
Haut-Katanga,8.0,10.0,7.0,2.0,27
Haut-Lomami,8.0,12.0,12.0,10.0,42
Haut-Uele,11.0,7.0,6.0,8.0,32
Ituri,2.0,5.0,4.0,2.0,13
Kasai,6.0,2.0,7.0,2.0,17
Kasai Central,2.0,7.0,1.0,5.0,15
Kasai Oriental,NaN,3.0,NaN,NaN,3


In [61]:
# Write the pivot table DataFrame to a CSV file
province_pts.to_csv('/home/sepal-user/module_results/esbae/DRC_province_esbae_points_count.csv', index=True)

In [48]:
poly_shp = poly_shp.to_crs("EPSG:3395")

In [49]:
#calculate area of poly
poly_shp['area_sqm'] = poly_shp.geometry.area

In [50]:
poly_shp['area_ha'] = poly_shp['area_sqm'] / 10000

In [51]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# Display the GeoDataFrame with the new area columns
print(poly_shp[['NOM', 'area_ha']])

               NOM  area_ha
0         Kinshasa  1058415
1    Kongo Central  5449333
2       Mai-Ndombe 12878817
3            Kwilu  7974749
4           Kwango  9110290
5         Equateur 10207799
6       Sud-Ubangi  5220098
7      Nord-Ubangi  5444914
8          Mongala  5625156
9          Tshuapa 13298111
10          Tshopo 20056004
11        Bas-Uele 14923195
12       Haut-Uele  9166490
13           Ituri  6283610
14       Nord-Kivu  5724051
15        Sud-Kivu  5931496
16         Maniema 12803642
17         Lualaba 12545160
18     Haut-Lomami 11137497
19       Tanganyka 12254047
20    Haut-Katanga 12652867
21  Kasai Oriental  1031690
22         Sankuru 10867105
23          Lomami  5461308
24   Kasai Central  5776514
25           Kasai  9748568


In [27]:
list_columns = joined_data.columns.tolist()

# Display the list of column names
print(list_columns)

['Unnamed: 0', 'point_id', 'chg_prob_cafi1520', 'def_prob_cafi1520', 'deg_prob_cafi1520', 'stable_prob_cafi1520', 'kmeans_chg_1520', 'kmeans_def_1520', 'kmeans_deg_1520', 'lon', 'lat', 'sampleid', 'sample_geom', 'Ref_FNF_2015', 'Ref_Ftype_2015', 'Ref_NFtype_2015', 'Ref_LCover_2015', 'Ref_Year_1522', 'Ref_Change_1522', 'Ref_Change_Year_1522', 'Ref_Change_Type_1522', 'Ref_Change_LCover_1522', 'Ref_Regeneration', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'IndMine', 'IndAg', 'ArtAg', 'Other', 'Other_Desc', 'collection', 'interpreted', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 

In [19]:
#drop any columns if needed
columns_to_drop = ['index_right', 'SCE_SEM', 'SCE_GEO', 'MODIF', 'ORIGINE', 'CODE_INS', 'SURFACE', 'CHEF_LIEU', 'ID_SEPAL', 'BLABLA']
joined_data = joined_data.drop(columns_to_drop, axis=1)

In [20]:
# Rename columns
column_mapping = {
   'NOM': 'Province'
#    'NAME_2': 'Territoire',
#    'Unnamed: 0':'Index'
}

# Use the rename() method to rename columns
joined_data.rename(columns=column_mapping, inplace=True)

In [21]:
out_file_shp_att =  '/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2023_province.csv'
joined_data.to_csv(out_file_shp_att,index=False)

#### get attributes from local raster

In [61]:
LC_map = "/home/sepal-user/data/lc_map/CAFI_LC_2015_19_wgs84.tif"

In [62]:
LC_src = rasterio.open(LC_map)

In [69]:
joined_data["CAFI_LC_2015"] = [x for x in LC_src.sample(coord_list)]
joined_data["CAFI_LC_2015"] = joined_data["CAFI_LC_2015"].str[0]
joined_data.head()

,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,Degall,Stable,NF,sampling,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,TropBiome,CAFI_LC_2015
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,1,0,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,1,0,0,str_random,AT0712,Northern Congolian Forest-Savanna Mosaic,12.0,AT0712. Northern Congolian forest-savanna mosaic,2.0,9
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,1,0,0,str_random,AT0705,East Sudanian Savanna,5.0,AT0705. East Sudanian savanna,2.0,11
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,0,1,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,11
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,0,1,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2


In [70]:
joined_data['CAFI_LC_2015'].value_counts(dropna=False)

CAFI_LC_2015
1     3345
4     1560
12    1150
3     1137
15     959
19     633
11     589
6      576
13     556
7      530
5      482
2      427
8      383
9      267
16     265
14     204
17      35
18      16
0        1
Name: count, dtype: int64

In [77]:
joined_columns = joined_data.columns.tolist()

# Display the list of column names
print(joined_columns)

['geometry', 'UniqueID', 'images', 'mon_images', 'bfast_change_date', 'bfast_magnitude', 'bfast_means', 'cusum_change_date', 'cusum_confidence', 'cusum_magnitude', 'red_mean', 'red_sd', 'red_min', 'red_max', 'nir_mean', 'nir_sd', 'nir_min', 'nir_max', 'swir1_mean', 'swir1_sd', 'swir1_min', 'swir1_max', 'swir2_mean', 'swir2_sd', 'swir2_min', 'swir2_max', 'ndfi_mean', 'ndfi_sd', 'ndfi_min', 'ndfi_max', 'brightness_mean', 'brightness_sd', 'brightness_min', 'brightness_max', 'greenness_mean', 'greenness_sd', 'greenness_min', 'greenness_max', 'wetness_mean', 'wetness_sd', 'wetness_min', 'wetness_max', 'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min', 'ewma_jrc_date', 'ewma_jrc_change', 'ewma_jrc_magnitude', 'mosum_jrc_date', 'mosum_jrc_change', 'mosum_jrc_magnitude', 'cusum_jrc_date', 'cusum_jrc_change', 'cusum_jrc_magnitude', 'ccdc_change_date', 'ccdc_magnitude', 'aspect', 'dw_class_mode', 'dw_tree_prob__max', 'dw_tree_prob__min', 'dw_tree_prob__stdDev', 'dw_tree_prob_mean', 

In [78]:
joined_data.head()

,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,sampling,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,TropBiome,CAFI_LC_2015,Frag_2015,Treecov_2015,Biomass_2010
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,5,52,94
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,str_random,AT0712,Northern Congolian Forest-Savanna Mosaic,12.0,AT0712. Northern Congolian forest-savanna mosaic,2.0,9,3,54,194
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,str_random,AT0705,East Sudanian Savanna,5.0,AT0705. East Sudanian savanna,2.0,11,2,33,9
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,11,2,28,52
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,3,51,87


#### get attributes from GEE asset

In [22]:
#path to asset
asset = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2022_2023_v2')

In [23]:
#resolution of asset
scale = 30

In [24]:
#name of band
band = 'strata'

In [25]:
#what to rename sampled band to. it is always called first
newName = 'TNT_newstrata'

In [26]:
#this can take a little while with many points
dfs = []
for i in range(0, len(points), 5000):
    tmp_df = gdf.loc[i:i+4999]
    tmp_fc = geemap.gdf_to_ee(tmp_df[['ID', 'TNT_stratum','geometry']])
    tmp_smp = asset.select(band).reduceRegions(**{
        'collection': tmp_fc,
        'scale': scale,
        'reducer': ee.Reducer.first()   
    })

    tmp_res = geemap.ee_to_gdf(tmp_smp)
    dfs.append(tmp_res)
    
sampled_df = pd.concat(dfs)

In [27]:
sampled_df

,geometry,ID,TNT_stratum,first
0,POINT (19.59266 -2.32390),0,2,1
1,POINT (23.35615 0.51523),1,2,1
2,POINT (23.85858 -6.53008),2,2,2
3,POINT (24.99800 -8.51401),3,2,0
4,POINT (23.00967 -0.68016),4,2,1
...,...,...,...,...
509,POINT (20.55287 2.52934),509,2,2
510,POINT (23.04111 -10.65577),510,2,2
511,POINT (28.38483 2.35013),511,2,2
512,POINT (22.54434 1.46098),512,2,1


In [28]:
sampled_df = sampled_df.rename(columns={'first':newName})
sampled_df

,geometry,ID,TNT_stratum,TNT_newstrata
0,POINT (19.59266 -2.32390),0,2,1
1,POINT (23.35615 0.51523),1,2,1
2,POINT (23.85858 -6.53008),2,2,2
3,POINT (24.99800 -8.51401),3,2,0
4,POINT (23.00967 -0.68016),4,2,1
...,...,...,...,...
509,POINT (20.55287 2.52934),509,2,2
510,POINT (23.04111 -10.65577),510,2,2
511,POINT (28.38483 2.35013),511,2,2
512,POINT (22.54434 1.46098),512,2,1


In [29]:
pd.pivot_table(sampled_df,values='ID',index=['TNT_stratum'],columns=['TNT_newstrata'],aggfunc="count",margins=True,
                             margins_name='Total')

TNT_newstrata,0,1,2,3,4,5,Total
TNT_stratum,,,,,,,
2,9.0,67.0,98.0,NaN,NaN,NaN,174
3,6.0,73.0,NaN,59.0,NaN,NaN,138
4,NaN,70.0,NaN,6.0,42.0,NaN,118
5,NaN,37.0,NaN,NaN,13.0,34.0,84
Total,15.0,247.0,98.0,65.0,55.0,34.0,514


#### export

In [31]:
out_file =  '/home/sepal-user/module_results/esbae/DRC/DRC_ceo_2023_prov_newstratum.csv'
sampled_df.to_csv(out_file,index=False)